In [1]:
def create_dataset():
    import numpy as np
    from PIL import Image
    import os
    import pathlib
    from hops import hdfs
    import shutil
    
    hdfs.ls(hdfs.project_path())
    hdfs.copy_to_local('TenGeoPSARwv/ML_PNG_SMALL_RESIZED', overwrite=True)
    data_dir = './ML_PNG_SMALL_RESIZED'
    data_dir = pathlib.Path(data_dir)
    CLASS_NAMES = np.array(sorted([item.name for item in data_dir.glob('train/*')]))

    train_image_path_list = list(data_dir.glob('train/*/*.png'))
    validation_image_path_list = list(data_dir.glob('validation/*/*.png'))
    test_image_path_list = list(data_dir.glob('test/*/*.png'))
    
    
    def get_np_dataset_from_path_list(path_list):
        X = []
        y = []
        for image_path in path_list:
            image = np.array(Image.open(image_path).getdata()).reshape((299,299))
            # image.shape
            image = np.stack([image, image, image], axis=-1)
            # image.shape
            label = str(image_path).split('/')[-2]
            one_hot_label = (label == CLASS_NAMES)
            X.append(image)
            y.append(one_hot_label)
        return np.asarray(X), np.asarray(y)
    
    X_train, y_train = get_np_dataset_from_path_list(train_image_path_list)
    X_validation, y_validation = get_np_dataset_from_path_list(validation_image_path_list)
    X_test, y_test = get_np_dataset_from_path_list(test_image_path_list)
    
    return X_train, y_train, X_validation, y_validation, X_test, y_test

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
111,application_1596125182098_0115,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [2]:
def base_model_generator():
    
    import tensorflow as tf
    

    my_inception = tf.keras.applications.inception_v3.InceptionV3()
    input_layer = my_inception.input
    hidden_layer = my_inception.layers[-2].output

    added_layer = tf.keras.layers.Dense(10, activation='softmax')(hidden_layer)
    cust_inception_v3 = tf.keras.Model(inputs=input_layer, outputs=added_layer, name='customized_inception_v3')
    
    return cust_inception_v3

In [3]:
def training_fn(dataset_function, model_function):

    import tensorflow as tf
    import datetime
    
    
    tf.enable_eager_execution()
    tf.reset_default_graph()
    
    tf.keras.backend.clear_session()

    tf.random.set_random_seed(1234)
    
    start_time = datetime.datetime.now()
    print("Trial started on: ", start_time)

    
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
      # Restrict TensorFlow to only use the first GPU
      try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        

        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
      except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

    # use this strategy to test your code before switching to other strategies which actually distributes to multiple devices/machines.
    strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")

    X_train, y_train, X_validation, y_validation, X_test, y_test = dataset_function()
    cust_inception_v3 = model_function()
    
    # compile the model
    cust_inception_v3.compile(optimizer='adam',
                          loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                          metrics=['accuracy'])

    history = cust_inception_v3.fit(X_train, y_train, epochs=40, validation_data=(X_validation, y_validation), verbose=0)
    test_score = cust_inception_v3.evaluate(X_test, y_test)
    
    print('Test loss:', test_score[0])
    print('Test accuracy:', test_score[1])
    
    tf.keras.backend.clear_session()

    end_time = datetime.datetime.now()
    
    print("Trial completed on: ", end_time)

    trial_time = end_time - start_time
    
    print("Trial duration: ", trial_time.total_seconds())
    
    tf.keras.backend.clear_session()
    tf.reset_default_graph()


    return test_score[1]

In [4]:
from maggy.ablation import AblationStudy

ablation_study = AblationStudy("TenGeoPSARwv", 1, "type",)
ablation_study.set_dataset_generator(create_dataset)
ablation_study.model.set_base_model_generator(base_model_generator)

ablation_study.model.add_module('mixed0', 'mixed1')
ablation_study.model.add_module('mixed2', 'mixed3')
ablation_study.model.add_module('mixed4', 'mixed5')

In [5]:
from maggy import experiment

result = experiment.lagom(map_fun=training_fn, experiment_type='ablation',
                           ablation_study=ablation_study, 
                           ablator='loco', 
                           name='Inception-3-modules-40-epochs-4-PARALLEL'
                          )

0: Trial started on:  2020-08-01 22:58:46.071190
0: Started copying hdfs://rpc.namenode.service.consul:8020/Projects/Inception_v3/TenGeoPSARwv/ML_PNG_SMALL_RESIZED to local disk on path /srv/hops/hopsdata/tmp/nm-local-dir/usercache/6B9maGNWZKY591Qiydpfx6wG5dR3t5XMM0HHltffC3g/appcache/application_1596125182098_0115/container_e01_1596125182098_0115_01_000002/

1: Trial started on:  2020-08-01 22:58:48.902350
1: Started copying hdfs://rpc.namenode.service.consul:8020/Projects/Inception_v3/TenGeoPSARwv/ML_PNG_SMALL_RESIZED to local disk on path /srv/hops/hopsdata/tmp/nm-local-dir/usercache/6B9maGNWZKY591Qiydpfx6wG5dR3t5XMM0HHltffC3g/appcache/application_1596125182098_0115/container_e01_1596125182098_0115_01_000003/

2: Trial started on:  2020-08-01 22:58:50.190705
2: Started copying hdfs://rpc.namenode.service.consul:8020/Projects/Inception_v3/TenGeoPSARwv/ML_PNG_SMALL_RESIZED to local disk on path /srv/hops/hopsdata/tmp/nm-local-dir/usercache/6B9maGNWZKY591Qiydpfx6wG5dR3t5XMM0HHltffC3g/ap

# The END